In [1]:
import os
import cv2
import time
from tqdm import tqdm
import pandas as pd
import warnings
warnings.simplefilter("ignore", category=pd.errors.SettingWithCopyWarning)

In [2]:
df = pd.read_csv('roi_distance.csv', sep=',')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [3]:
df.head()

,xcenter,ycenter,width,height,confidence,class,name,area,depth_id,depth_name,image_name
0,354.030640,680.527527,116.334442,126.692017,0.917122,4,yellow_cone,14738.645075,0,close_from_cam,gfr_00544_jpg.jpg
1,1556.451660,912.496704,143.378784,190.541870,0.911995,0,blue_cone,27319.661673,0,close_from_cam,gfr_00544_jpg.jpg
2,688.939575,758.156433,119.413635,144.132568,0.901687,2,orange_cone,17211.393946,0,close_from_cam,gfr_00544_jpg.jpg
3,1153.819580,710.862671,111.585693,122.757141,0.900037,4,yellow_cone,13697.940706,0,close_from_cam,gfr_00544_jpg.jpg
4,497.007996,550.327332,67.068726,76.685883,0.895754,0,blue_cone,5143.224414,0,close_from_cam,gfr_00544_jpg.jpg


In [4]:
def ground_truth_df(image_name_list):
    final_df = pd.DataFrame()
    for image_name in tqdm(image_name_list):
        txt_file = os.path.basename(image_name[:-4])+'.txt'
        file_path = f'./data/labels/{txt_file}'
        image_path = f'./data/images/{image_name}'

        df_txt = pd.read_csv(file_path, sep=' ', header=None, names=['class', 'xcenter', 'ycenter', 'width', 'height'])
        df_txt['image_name'] = image_path
        df_txt['label_name'] = file_path
        final_df = pd.concat([final_df, df_txt])
    return final_df

In [5]:
image_name_list = df.image_name.unique()
df_truth = ground_truth_df(image_name_list)
df_truth = df_truth.reset_index(drop=True)

100%|████████████████████████████████████| 11923/11923 [01:02<00:00, 191.74it/s]


In [6]:
df_truth

,class,xcenter,ycenter,width,height,image_name,label_name
0,0,0.263074,0.369128,0.035922,0.051007,./data/images/gfr_00544_jpg.jpg,./data/labels/gfr_00544_jpg.txt
1,0,0.822240,0.611409,0.074485,0.128859,./data/images/gfr_00544_jpg.jpg,./data/labels/gfr_00544_jpg.txt
2,1,0.327258,0.413087,0.056524,0.118792,./data/images/gfr_00544_jpg.jpg,./data/labels/gfr_00544_jpg.txt
3,1,0.105652,0.320470,0.042261,0.073154,./data/images/gfr_00544_jpg.jpg,./data/labels/gfr_00544_jpg.txt
4,2,0.398838,0.387248,0.030639,0.055034,./data/images/gfr_00544_jpg.jpg,./data/labels/gfr_00544_jpg.txt
...,...,...,...,...,...,...,...
227862,3,0.807273,0.534302,0.002727,0.006977,./data/images/ff_00354.png,./data/labels/ff_00354.txt
227863,3,0.721818,0.542442,0.005455,0.010465,./data/images/ff_00354.png,./data/labels/ff_00354.txt
227864,3,0.699773,0.538372,0.003182,0.008140,./data/images/ff_00354.png,./data/labels/ff_00354.txt
227865,3,0.745455,0.537209,0.003636,0.006977,./data/images/ff_00354.png,./data/labels/ff_00354.txt


In [7]:
def get_image_shape(image_name):
    image = cv2.imread(image_name)
    image_width, image_height = image.shape[1], image.shape[0]
    return image_width, image_height
    

def get_shape(df_truth):
    time_interval = 1
    unique_image_name_list = df_truth.image_name.unique()
    final_df = pd.DataFrame()

    for image_name in tqdm(unique_image_name_list):
        image_width, image_height = get_image_shape(image_name)
        time.sleep(time_interval) # WAIT 1 Sec before processing next
        
        condition = (df_truth['image_name'] == image_name)
        sliced_df = df_truth[condition]
        sliced_df['image_width'] = image_width
        sliced_df['image_height'] = image_height    
        final_df = pd.concat([final_df, sliced_df])
    return final_df

In [8]:
df_final_truth = get_shape(df_truth)

100%|███████████████████████████████████| 11923/11923 [3:35:47<00:00,  1.09s/it]


In [10]:
df_final_truth.to_csv('ground_truth.csv', sep=',')